## Day 17: Pyroclastic Flow

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day17/puzzle.ipynb)

### Part 1

In [1]:
type JetsState = { Pattern: char array; Index: int }
with member this.Next () = 
        { this with Index = (this.Index + 1) % this.Pattern.Length }
     member this.Current = this.Pattern[this.Index]


It might be reasonable to model figures and the chamber itself as binary masks, which promises to be easy to detect collisions of and to shift by jets.

In [2]:
type Figure = uint8 list

let inline (>>) figure shift = List.map (fun y -> y >>> shift) figure
let inline (<<) figure shift = List.map (fun y -> y <<< shift) figure

In [3]:
let figures = [|
    [ 
        0b0011110uy
    ]
    [
        0b0001000uy
        0b0011100uy
        0b0001000uy
    ]
    [
        0b0000100uy
        0b0000100uy
        0b0011100uy
    ]
    [
        0b0010000uy
        0b0010000uy
        0b0010000uy
        0b0010000uy
    ]
    [
        0b0011000uy
        0b0011000uy
    ]
|] 


type FiguresState = { Figures: Figure array; Index : int }
with member this.Next() = 
        { this with Index = (this.Index + 1) % this.Figures.Length }
     member this.Current = this.Figures[this.Index]

In [4]:
type FallingState = { Figure : Figure; Position: int }

type State = { 
    RestedFiguresCount: int64
    TowerHeight: int64
    Rested: Figure; 
    Figures: FiguresState
    Jets: JetsState;
    Falling: FallingState
}


In [5]:
#load "../common/common.fsx"

let stateDefaultSample = 
    let jets = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>" |> Seq.toArray
    
    {
        Rested = [
            0uy
            0uy
            0uy
            0uy
        ]
        RestedFiguresCount = 0
        TowerHeight = 0
        Figures = { Figures = figures; Index = 0}
        Jets = { Pattern = jets; Index = 0}
        Falling = { Figure = figures[0]; Position = 0 }
    }

In [6]:
let jet (state : State) = 
    let { Falling = falling } = state
    let newFalling = 
        match state.Jets.Current with
        | '>' -> 
            if (List.exists (fun x -> x &&& 0b0000001uy <> 0uy) falling.Figure)
            then falling
            else {falling with Figure = falling.Figure >> 1}
        | '<' -> 
            if (List.exists (fun x -> x &&& 0b1000000uy <> 0uy) falling.Figure)
            then falling
            else {falling with Figure = falling.Figure << 1}
    { state with Falling = newFalling }

In [7]:
let gravity (state : State) = 
    { state with Falling = { state.Falling with Position = state.Falling.Position + 1} }

In [8]:
let private revertIfCollision (state : State) (requestedState : State) = 
    let { Falling = { Figure = figure; Position = pos } } = requestedState
    let figureHeight = List.length figure
    if (List.length state.Rested - pos < figureHeight)
    then state
    else 
        let collision = 
            state.Rested |> List.skip pos |> List.take figureHeight
            |> List.zip figure
            |> List.map (fun (x,y) -> x &&& y)
            |> List.exists ((<>)0uy)
        if collision then state
        else requestedState

let checkCollision (f : State -> State) = 
    fun state ->
        let newState = f state
        revertIfCollision state newState

In [9]:

let fix state =
    let { Falling = { Figure = figure; Position = pos } } = state
    let figureHeight = List.length figure
    let (before, rest1) = List.splitAt pos state.Rested
    let (atFalling, rest) = List.splitAt figureHeight rest1

    let newRested = 
        atFalling
        |> List.zip figure
        |> List.map (fun (x,y) -> x ||| y)

    let heightIncrease = 
        atFalling |> List.filter ((=)0uy) |> List.length

    {
        state with Rested = before @ newRested @ rest; 
                TowerHeight = state.TowerHeight + int64 heightIncrease;
                RestedFiguresCount = state.RestedFiguresCount + 1L; 
    }

In [10]:
let newFigure s = 
    let figures = s.Figures.Next();
    
    let newRested = 
        s.Rested
        |> List.skipWhile ((=)0uy)
        |> List.append (List.replicate (figures.Current.Length + 3) 0uy)
    { s with Rested = newRested; Figures = figures; Falling = { Figure = figures.Current; Position = 0 }}

In [11]:
let nextState state = 
    let afterJet = state |> checkCollision jet
    let afterGravity = afterJet |> checkCollision gravity
    
    if (afterJet.Falling = afterGravity.Falling) 
    then 
        { (afterJet |> fix |> newFigure) with Jets = state.Jets.Next(); }
    else
        { afterGravity with Jets = state.Jets.Next()}

In [12]:
let infiniteStates initial =
    Seq.unfold (fun s -> let newS = nextState s in Some(newS, newS)) initial


In [127]:
#!time

(infiniteStates stateDefaultSample
|> Seq.find (fun s -> s.RestedFiguresCount = 2022L)
).TowerHeight

3068

Wall time: 533.7298ms

In [14]:
#!value --name actualRaw --from-file ./data_actual.txt

In [15]:
#!share actualRaw --from value
let stateDefaultActual = 
    let jets = actualRaw |> Seq.toArray
    
    {
        Rested = [
            0uy
            0uy
            0uy
            0uy
        ]
        RestedFiguresCount = 0
        TowerHeight = 0
        Figures = { Figures = figures; Index = 0}
        Jets = { Pattern = jets; Index = 0}
        Falling = { Figure = figures[0]; Position = 0 }
    }

In [126]:
#!time
(infiniteStates stateDefaultActual
|> Seq.find (fun s -> s.RestedFiguresCount = 2022L)
).TowerHeight

3130

Wall time: 868.3351ms

### Part 2

For Part 2 the straightforward algorithm is going to take a while. On the other hand, the process promises to be periodic, and we can try to identify the period. In other words, how many figures should fall to get back to the same input state:
1. Jets state
2. Figures state
3. Rested figures pattern

For the latter we can consider some reasonable number of top lines, say, 50.

In [99]:
type RepetitionState = { Rested: Figure; JetsIndex: int; FiguresIndex: int}

let private repetitionsState (state : State) = 
    { Rested = state.Rested |> List.truncate 50; JetsIndex = state.Jets.Index; FiguresIndex = state.Figures.Index }

type Repetition = { Start: int; StartState: State; Period: int; NextState: State}

open System.Collections.Generic
let findRepetition (states: seq<State>) = 
    let cache = Dictionary<_, _>()
    states
    |> Seq.distinctBy (fun s -> s.RestedFiguresCount)
    |> Seq.choose (fun state -> 
        let key = repetitionsState state
        match cache.TryGetValue(key) with
        | false, _ -> 
            cache.Add(key, state)
            None
        | true, value -> 
            Some { Start = int value.RestedFiguresCount; StartState = value; Period = int state.RestedFiguresCount - int value.RestedFiguresCount; NextState = state }
    ) |> Seq.head


In [100]:
infiniteStates stateDefaultSample
|> findRepetition

Start,StartState,Period,NextState
42,"State RestedFiguresCount: 42 TowerHeight: 69 Rested: [ 0, 0, 0, 0, 0, 0, 32, 112, 62, 56, 56, 40, 42, 63, 62, 4, 4, 4, 4, 52 ... (more) ] Figures: FiguresState Figures: FSharpList<Byte>[] [ 30 ] [ 8, 28, 8 ] [ 4, 4, 28 ] [ 16, 16, 16, 16 ] [ 24, 24 ] Index: 2 Current: [ 4, 4, 28 ] Jets: JetsState Pattern: [ >, >, >, <, <, >, <, >, >, <, <, <, >, >, <, >, >, >, <, < ... (20 more) ] Index: 30 Current: < Falling: FallingState Figure: [ 4, 4, 28 ] Position: 0",35,"State RestedFiguresCount: 77 TowerHeight: 122 Rested: [ 0, 0, 0, 0, 0, 0, 32, 112, 62, 56, 56, 40, 42, 63, 62, 4, 4, 4, 4, 52 ... (more) ] Figures: FiguresState Figures: FSharpList<Byte>[] [ 30 ] [ 8, 28, 8 ] [ 4, 4, 28 ] [ 16, 16, 16, 16 ] [ 24, 24 ] Index: 2 Current: [ 4, 4, 28 ] Jets: JetsState Pattern: [ >, >, >, <, <, >, <, >, >, <, <, <, >, >, <, >, >, >, <, < ... (20 more) ] Index: 30 Current: < Falling: FallingState Figure: [ 4, 4, 28 ] Position: 0"


In [101]:
infiniteStates stateDefaultActual
|> findRepetition

Start,StartState,Period,NextState
417,"State RestedFiguresCount: 417 TowerHeight: 648 Rested: [ 0, 0, 0, 0, 0, 0, 32, 112, 32, 120, 8, 8, 24, 24, 120, 28, 8, 60, 100, 100 ... (more) ] Figures: FiguresState Figures: FSharpList<Byte>[] [ 30 ] [ 8, 28, 8 ] [ 4, 4, 28 ] [ 16, 16, 16, 16 ] [ 24, 24 ] Index: 2 Current: [ 4, 4, 28 ] Jets: JetsState Pattern: [ >, >, <, <, <, >, >, >, >, <, <, <, >, >, <, >, >, >, >, < ... (10071 more) ] Index: 2533 Current: < Falling: FallingState Figure: [ 4, 4, 28 ] Position: 0",1725,"State RestedFiguresCount: 2142 TowerHeight: 3333 Rested: [ 0, 0, 0, 0, 0, 0, 32, 112, 32, 120, 8, 8, 24, 24, 120, 28, 8, 60, 100, 100 ... (more) ] Figures: FiguresState Figures: FSharpList<Byte>[] [ 30 ] [ 8, 28, 8 ] [ 4, 4, 28 ] [ 16, 16, 16, 16 ] [ 24, 24 ] Index: 2 Current: [ 4, 4, 28 ] Jets: JetsState Pattern: [ >, >, <, <, <, >, >, >, >, <, <, <, >, >, <, >, >, >, >, < ... (10071 more) ] Index: 2533 Current: < Falling: FallingState Figure: [ 4, 4, 28 ] Position: 0"


In [120]:
let stateAt initState repetitionState restedFiguresCount = 
    if (restedFiguresCount <= repetitionState.NextState.RestedFiguresCount)
    then 
        infiniteStates initState
        |> Seq.find (fun p -> p.RestedFiguresCount = restedFiguresCount)
    else 
        let heightInPeriod = repetitionState.NextState.TowerHeight - repetitionState.StartState.TowerHeight
        let periods = (restedFiguresCount - repetitionState.StartState.RestedFiguresCount) / (int64 repetitionState.Period)
        let s = {
            repetitionState.StartState with 
                RestedFiguresCount = periods * (int64 repetitionState.Period) + repetitionState.StartState.RestedFiguresCount
                TowerHeight = periods * heightInPeriod + repetitionState.StartState.TowerHeight
        }
        infiniteStates s
        |> Seq.find (fun p -> p.RestedFiguresCount = restedFiguresCount)

In [121]:
#!time
stateAt stateDefaultSample (infiniteStates stateDefaultSample |> findRepetition) 1000000000000L

RestedFiguresCount,TowerHeight,Rested,Figures,Jets,Falling
1000000000000,1514285714288,"[ 0, 0, 0, 0, 28, 28, 30, 7, 2, 62, 36, 36, 61, 61, 119, 62, 56, 56, 40, 42 ... (more) ]","FiguresState Figures: FSharpList<Byte>[] [ 30 ] [ 8, 28, 8 ] [ 4, 4, 28 ] [ 16, 16, 16, 16 ] [ 24, 24 ] Index: 0 Current: [ 30 ]","JetsState Pattern: [ >, >, >, <, <, >, <, >, >, <, <, <, >, >, <, >, >, >, <, < ... (20 more) ] Index: 2 Current: >",FallingState Figure: [ 30 ] Position: 0


Wall time: 28.0351ms

In [125]:
#!time
stateAt stateDefaultActual (infiniteStates stateDefaultActual |> findRepetition) 1000000000000L

RestedFiguresCount,TowerHeight,Rested,Figures,Jets,Falling
1000000000000,1556521739139,"[ 0, 0, 0, 0, 24, 24, 16, 16, 20, 20, 28, 16, 56, 16, 30, 4, 100, 102, 38, 126 ... (more) ]","FiguresState Figures: FSharpList<Byte>[] [ 30 ] [ 8, 28, 8 ] [ 4, 4, 28 ] [ 16, 16, 16, 16 ] [ 24, 24 ] Index: 0 Current: [ 30 ]","JetsState Pattern: [ >, >, <, <, <, >, >, >, >, <, <, <, >, >, <, >, >, >, >, < ... (10071 more) ] Index: 9394 Current: >",FallingState Figure: [ 30 ] Position: 0


Wall time: 907.9573ms